In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import sqlalchemy as sa

load_dotenv()

engine: sa.Engine = sa.create_engine(os.getenv("URL_MYSQL"))

In [ ]:
stmt: str = """
    CREATE TABLE IF NOT EXISTS unibb (
        id INT AUTO_INCREMENT PRIMARY KEY,
        id_curso INT NOT NULL,
        nm_curso VARCHAR(100) NOT NULL,
        hr_curso TINYINT UNSIGNED NOT NULL
    )
"""

with engine.begin() as cnx:
    cnx.execute(sa.text(stmt))

"Tabela criada com sucesso."

In [ ]:
try:
    df_new: pd.DataFrame = pd.read_csv("./src/unibb.csv", sep=",", encoding="utf-8-sig")
    df_new = df_new[df_new["save"].eq(1)]
    df_new = df_new[["id_curso", "nm_curso", "hr_curso"]]
    rows_inserted: int = df_new.to_sql(name="unibb", con=engine, if_exists="append", index=False)

    f"Foram {rows_inserted} cursos inseridos com sucesso."
except FileNotFoundError:
    "Erro ao localizar o arquivo .CSV..."
except UnicodeEncodeError:
    "Erro ao decodificar o arquivo .CSV..."
else:
    df_deleted: pd.DataFrame = pd.read_csv("./src/unibb.csv", sep=",", encoding="utf-8-sig")
    df_deleted[df_deleted["save"].ne(1)].to_csv("./src/unibb.csv", index=False, sep=",", encoding="utf-8-sig")

    "Cursos atualizados com sucesso."

In [ ]:
# classificar por ID
stmt: str = "SELECT id_curso AS Código, nm_curso AS Curso, hr_curso AS Horas FROM unibb ORDER BY id"

pd.read_sql(sql=sa.text(stmt), con=engine)

In [ ]:
# classificar por ID_CURSO
stmt: str = "SELECT id_curso AS Código, nm_curso AS Curso, hr_curso AS Horas FROM unibb ORDER BY id_curso"

pd.read_sql(sql=sa.text(stmt), con=engine)

In [ ]:
# verificar cursos duplicados
stmt: str = """
    SELECT id_curso AS Código, nm_curso AS Curso, hr_curso AS Horas FROM unibb WHERE nm_curso IN
        (SELECT nm_curso FROM unibb GROUP BY nm_curso HAVING COUNT(nm_curso) > 1)
    ORDER BY nm_curso, id_curso
"""

pd.read_sql(sql=sa.text(stmt), con=engine)